This notebook implements the **A* search** algorithm also known as informed search strategy. This is most widely known best form of best first search.It opens nodes based on the value of evaluation function which is sum of actual cost function and hueristic function.For the cheapest solution expand the node with lowest value of evaluation function. This algo is both complete and optimal.

Some real life application of algo:
1. Traffic managaemet 
2. Supply chain optimization
3. Maze solving

In [8]:
# Importing required modules.
import pandas as pd # importing pandas 
heuristic_data = pd.read_csv('hueristic_distances.csv') # reading csv file.
cities_data = pd.read_csv('connected_cities.csv')

In [9]:
# Creating graph class.
class Graph(object):
    '''This is a graph.'''
    def __init__(self, dataset):
        '''Initializes the graph.'''
        self.graph = self._create_graph(dataset)

    # Accesor methods
    @property
    def graph_data(self):
        '''Return the adjacency list of the nodes.'''
        return self.graph
    
    @property
    def total_nodes(self):
        '''The total unique nodes in the graph.'''
        total_node = 0
        for i in self.graph:  # iterating over each key in dictinary.
            total_node += 1
        return total_node
    
    @property
    def edges(self):
        '''Total number of edges in the graph. Since the graph is undirected so there
        is one edge between any two nodes of the graph.'''
        total_edges = 0
        for i in self.graph:
            total_edges += len(self.graph[i])  #Calcuting length of each list for every node.
        return total_edges
    
    def is_connected(self):
        '''
        Return True if graph is connected otherwise False.
        
        Since each node of the graph is considered as key in the adjacency list(actually a dictionary).
        So, if any key is having value as blank list then the node is isolated from the graph. Since
        here in the graph the list contains all the nodes to which key node is connnected. So, if this
        is the case then graph is disconnected.

        Note: Connected does not mean fully connected. This function only tell whether a graph is 
        connected or not.
        '''
        for i in self.graph:
            if len(self.graph[i]) == 0:
                return False
        return True
    
    #Non-public helper functions.
    def _create_graph(self, dataset):
        '''This is helper function to create graph.'''
        graph = {}
        for node1, node2 in zip(dataset['node1'], dataset['node2']):
            if node1 not in graph:
                graph[node1] = []   #Adding nodes to the graph.
            graph[node1].append(node2)  #Adding edges to the graph.
        return graph

    def heuristic_function(self, df):
        '''Returns heuristic distances for all the nodes.'''
        heuristic_distances = {}        # storing hueristic distance of each other cities.
        for node1, node2, hueristic, route in zip(df['node1'], df['node2'], df['Heuristic distance'], df['route_distance']):
            if node1 not in heuristic_distances:
                heuristic_distances[node1] = {}
            heuristic_distances[node1][node2] = (hueristic, route)    # adding the data to dictionary.
        return heuristic_distances
    
    
# Graph creation.
graph = Graph(cities_data)
heuristic_distances = graph.heuristic_function(heuristic_data)

#------------------Checking some information about grapah.---------------------------
print(graph.graph_data) #giving details of nodes connected to any node.
print('Total nodes in the graph: ', graph.total_nodes)
print('Total edges in the graph: ', graph.edges)
print('Is graph conneced: ', graph.is_connected())

{'Jodhpur': ['Bikaner', 'Rajsamand'], 'Rajsamand': ['Jodhpur', 'Sikar'], 'Bikaner': ['Jodhpur', 'Sri Ganganagar'], 'Sri Ganganagar': ['Bikaner', 'Sikar'], 'Sikar': ['Sri Ganganagar', 'Rajsamand', 'Una', 'Jaipur'], 'Una': ['Sikar', 'Baghpat'], 'Jaipur': ['Bundi', 'Delhi', 'Sikar'], 'Bundi': ['Jaipur', 'Kota', 'Belagavi'], 'Belagavi': ['Bundi', 'Hanamkonda', 'Calicut'], 'Calicut': ['Belagavi'], 'Delhi': ['Jaipur', 'Faridabad', 'Baghpat'], 'Kota': ['Bundi', 'Bhopal', 'Agra'], 'Baghpat': ['Una', 'Delhi', 'Aligarh'], 'Faridabad': ['Agra', 'Delhi'], 'Bhopal': ['Kota', 'Morena'], 'Agra': ['Faridabad', 'Aligarh', 'Morena', 'Kota'], 'Aligarh': ['Agra', 'Baghpat', 'Sitapur', 'Mahoba'], 'Morena': ['Bhopal', 'Sagar', 'Agra'], 'Sagar': ['Morena', 'Balaghat'], 'Balaghat': ['Sagar', 'Hanamkonda'], 'Hanamkonda': ['Balaghat', 'Belagavi'], 'Mahoba': ['Aligarh', 'Lucknow', 'Chitrakoot'], 'Sitapur': ['Aligarh', 'Lucknow'], 'Lucknow': ['Sitapur', 'Mahoba', 'Raebareli', 'Lakhimpur'], 'Lakhimpur': ['Lucknow'

In [10]:
#Implementing the A_star algorithm.
visited_list = []           # nodes visited in tree.
path = []                   # path
parent_child_mapping = {}   # parent child relationship for nodes.
evaluated_value = {}        # the evaluation function value for nodes

def a_star_search(start_node, goal_node):
    '''Returns the path between start node and goal node.'''
    if start_node not in visited_list:
        visited_list.append(start_node)
        if start_node == goal_node:
            path_construct(goal_node)   #constructing path.
            return True
        for node in graph.graph[start_node]:
            if node not in visited_list and node not in parent_child_mapping:
                parent_child_mapping[node] = start_node # creating parent child dictionary.
            if node not in visited_list:
                eval_value = heuristic_distances[node][goal_node][0] + heuristic_distances[visited_list[0]][node][1]
                evaluated_value[node] = eval_value  #finding values of evaluation function and storing.
        start_node = find_start_node()  #node for expansion with minimum evaluated value.
        del evaluated_value[start_node]
        a_star_search(start_node, goal_node)

def find_start_node():
    '''Returns  start node on the basis of minimum evaluated value 
    of evaluation of functions.'''
    min = 100000000000000   #assuming all distances are lesser than this.
    for key in evaluated_value:
        if evaluated_value[key] < min:
            min = evaluated_value[key]
            start_node = key
    return start_node

def path_construct(node):
    '''Return the path from stating node to goal node.'''
    node = node
    while node in parent_child_mapping: # iterating while there is parent for nodes.
        path.insert(0, node)
        node = parent_child_mapping[node]
    path.insert(0, node)


#------------checking output for different values of start node and goal node----------------------
cases = {1: ['Bikaner', 'Patna'], 2: ['Sagar', 'Agra'], 3: ['Sikar', 'Gaya'], 4: ['Calicut', 'Bhopal'], 5:['Bhopal', 'Calicut'], 6: ['Agra', 'Agra']}
for case in cases:
    a_star_search(cases[case][0], cases[case][1])
    print(f'The path from {cases[case][0]} to {cases[case][1]} is: ', path)
    path = []
    visited_list = []
    parent_child_mapping = {}
    evaluated_value = {}

The path from Bikaner to Patna is:  ['Bikaner', 'Sri Ganganagar', 'Sikar', 'Jaipur', 'Delhi', 'Faridabad', 'Agra', 'Aligarh', 'Sitapur', 'Lucknow', 'Lakhimpur', 'Arrah', 'Rohtas', 'Daudnagar', 'Patna']
The path from Sagar to Agra is:  ['Sagar', 'Morena', 'Agra']
The path from Sikar to Gaya is:  ['Sikar', 'Jaipur', 'Delhi', 'Faridabad', 'Agra', 'Aligarh', 'Sitapur', 'Lucknow', 'Raebareli', 'Prayagraj', 'Mirzapur', 'Ghazipur', 'Rohtas', 'Daudnagar', 'Patna', 'Jehanabad', 'Nawada', 'Gaya']
The path from Calicut to Bhopal is:  ['Calicut', 'Belagavi', 'Bundi', 'Kota', 'Bhopal']
The path from Bhopal to Calicut is:  ['Bhopal', 'Kota', 'Bundi', 'Belagavi', 'Calicut']
The path from Agra to Agra is:  ['Agra']


# Observation
1. Always return a path if nodes are connected.
2. Always gives shortest path.
3. Always return the same path length even if start node and goal node are just interchanged.
4. Always return the same path nodes even if start node and goal node are just interchanged.
